# Scraping Boxofficemojo

### Scraping studio links

In [ ]:
# Imports
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import scraping_class

# Get soup
logfile = 'sds19exam2.csv'
connector = scraping_class.Connector(logfile)
response, my_id = connector.get('https://www.boxofficemojo.com/studio/', 'scraping-bom')
html = response.text
soup = bs(html,'lxml')

# Scrape list of studios
table_node = soup.find_all('table', attrs = {'border': '0', 'cellspacing': '1', 'cellpadding': '5'})
link_nodes = soup.find_all('a', href = True)
studios = []
for link in link_nodes:
    if '/studio/chart/' in link['href']:
        studios.append(link['href'].split('&studio=')[1].split('.htm')[0])

### Function for scraping a studio's movies

In [ ]:
def get_movies(studio):
    i = 1
    while True:
        # Get HTML
        logfile = 'sds19exam2.csv'
        connector = scraping_class.Connector(logfile)
        response, my_id = connector.get('https://www.boxofficemojo.com/studio/chart/?page=' + str(i) + '&yr=&view=company&view2=allmovies&studio=' + studio + '.htm&p=.htm', 'scraping-bom')
        html = response.text
        tables = pd.read_html(html, header = 0)
        
        # Get dataframe
        if len(tables) == 2: break
        dfs = tables[1]
        dfs.drop(dfs.tail(4).index, inplace = True)
        dfs.columns = ['boRank', 'title', 'studioAcronym', 'boWorldwide', 'totalTheaters', 'boOpening', 'openingTheaters', 'openDate'] 
        
        # Append dataframe
        if i == 1:
            dfn = dfs.copy()
        else:
            dfn = dfn.append(dfs)
            
        # Next loop
        i += 1
        time.sleep(1)
    try:
        return dfn
    except:
        return None

### Scraping all movies

In [ ]:
i = 1
for studio in studios:
    # Append movies
    if i == 1:
        dff = get_movies(studio)
    else:
        try:
            dff = dff.append(get_movies(studio))
        except:
            pass
        
    # Print iteration number
    print(i)
    
    # Next loop
    i += 1
    time.sleep(1)

### Structuring data

In [ ]:
# Function for converting string to year
def convert_year(string):
    try:
        string = string.split('/')[2]
        if string[0] in '23456789':
            return int('19' + string)
        else:
            return int('20' + string)
    except:
        return None

# Function for turning amount of money with ',' and '$' into a number
def numerize_money(string):
    if isinstance(string, str):
        try:
            return int(float(string.replace('$', '').replace(',', '')))
        except:
            return None
    else:
        return None

# Apply functions
dff['boOpening'] = dff['boOpening'].apply(numerize_money)
dff['boWorldwide'] = dff['boWorldwide'].apply(numerize_money)
dff['year'] = dff['openDate'].apply(convert_year)
dff['year'] = dff['openDate'].apply(convert_year)
dff = dff.drop('openDate', axis = 1)

# Save dataframe
dff.to_pickle('bom_clean.pkl')